### Data Preparation

In [1]:
NUM_OF_LABELS = 9
BATCH_SIZE = 32

In [2]:
import tensorflow as tf
from Load_Data import load_image_dataset_from_directory, load_image_batch_size_one

# filepaths
b_dir = '/datasets/small_datasets/coffee'
#b_dir = '/datasets/rock_paper_scissor_original'
tr_dir = '/train'
val_dir = '/train'
ts_dir = '/test'

# loading data
train_dataset, valid_dataset, test_dataset = load_image_dataset_from_directory(b_dir, tr_dir, val_dir, ts_dir)

# loading data/
# label_names=['바리스타룰스 쇼콜라모카', '바리스타룰스 에스프레소 라떼', '티오피 마일드',
#              '바리스타룰스 모카프레소', '바리스타룰스 카라멜 딥 프레소', '스타벅스 스키니라떼',
#              '카페라떼 마일드', '스페셜티카페라떼 마일드', '바리스타룰스 로 슈거 에스프레소 라떼']
# training_dataset = load_image_batch_size_one(b_dir, tr_dir, label_names=label_names)
# validation_size = int(0.2*len(training_dataset))
# train_dataset = training_dataset[:-validation_size]
# valid_dataset = training_dataset[-validation_size:]
# test_dataset = load_image_batch_size_one(b_dir, ts_dir, label_names=label_names)

c:\Users\Programming\anaconda3\envs\c_project\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Model(VGG19+SPP)

* Spatial_Pyramid_Pooling(https://arxiv.org/abs/1406.4729) code is provided by yhenon(https://github.com/yhenon/keras-spp)

In [3]:
import os, sys
sys.path.append(os.getcwd()+'/sppnet')
from SpatialPyramidPooling import SpatialPyramidPooling

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Normalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense



def make_model():
    model = Sequential()

    #model.add(Normalization())
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(strides=(2,2)))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(strides=(2,2)))

    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(strides=(2,2)))

    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(strides=(2,2)))

    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(SpatialPyramidPooling([1,2,4]))
    #model.add(MaxPooling2D(strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(units=4096, activation='relu', ))
    model.add(Dense(units=4096, activation='relu', input_shape=(None,4096) ))
    model.add(Dense(units=NUM_OF_LABELS, activation='softmax', input_shape=(None,4096) ))
    
    return model

In [5]:
def compile_model(model):
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy']
                )

### Model Training

In [6]:
import numpy as np

model = make_model()
compile_model(model)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="VGG19",
        save_best_only=True,
        monitor="val_loss"
    )
]

history = model.fit(
    x = train_dataset,
    epochs=100,
    validation_data=valid_dataset,
    callbacks=callbacks,
    use_multiprocessing=True,
    )
model.summary()

ValueError: Data is expected to be in format `x`, `(x,)`, `(x, y)`, or `(x, y, sample_weight)`, found: ([<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=340x393 at 0x17D9F835AF0>, 7], [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=340x255 at 0x17D9F835B50>, 7], [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=340x457 at 0x17D9F835BB0>, 7], [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=340x255 at 0x17D9F835C10>, 7], [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=340x255 at 0x17D9F835C70>, 7], [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=340x340 at 0x17D9F835CD0>, 7], [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=340x535 at 0x17D9F835D30>, 7], [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=340x453 at 0x17D9F835D90>, 7], [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=340x264 at 0x17D9F835DF0>, 7], [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=340x255 at 0x17D9F835E50>, 7], [<PIL.JpegImagePlugin.Jpeg

+ WARNING:absl:Found untraced functions such as ... while saving. These functions will not be directly callable after loading.
: ModelCheckpoint를 사용하는 과정에서 발견되는 오류. tf.saved_model.save 대신 model.save를 사용할 것을 권장하기 위해 출력시키는 경고라는 의견이 있음.
(https://github.com/tensorflow/tensorflow/issues/47479#issuecomment-815314034)

### Prediction Accuracy

In [ ]:
model = keras.models.load_model("VGG19")
print(f"테스트 정확도: {model.evaluate(test_dataset)[1]:.3f}")

15/15 [==============================] - 3s 181ms/step - loss: 2.1972 - accuracy: 0.1156
테스트 정확도: 0.116


In [ ]:
y_predict = model.predict(test_dataset)
predicted_labels = y_predict.argmax(axis=-1)
print(f"테스트 정확도: {model.evaluate(test_dataset)[1]}")
print(f"예측 라벨: {predicted_labels}")
print(f"예측 정도: {model.predict(test_dataset)}")
print(f"{len(predicted_labels)} data are predicted.")

15/15 [==============================] - 2s 144ms/step - loss: 2.1972 - accuracy: 0.1156
테스트 정확도: 0.11555555462837219
예측 라벨: [6 8 6 8 6 8 6 8 6 6 6 8 8 8 8 6 6 8 8 8 8 6 8 8 8 8 8 8 6 8 6 8 8 8 8 6 6
 8 8 8 8 8 8 6 8 8 8 8 6 6 8 8 8 6 8 8 6 8 8 8 8 8 8 8 6 8 8 6 8 6 6 8 8 8
 8 8 8 6 8 8 8 8 6 6 8 8 6 8 8 8 8 6 6 8 6 8 8 8 8 8 8 8 6 8 8 8 8 8 8 6 6
 8 6 6 8 8 6 8 8 8 8 6 8 8 8 8 6 8 8 8 8 8 8 6 6 6 6 8 8 8 8 8 8 6 8 6 8 8
 8 8 8 6 6 6 8 8 8 8 8 6 6 8 8 6 8 6 8 6 8 8 8 8 8 8 6 8 8 8 8 6 8 8 8 8 8
 6 8 6 8 6 8 6 8 8 8 8 8 8 8 6 6 8 8 8 8 6 8 8 8 8 6 8 6 8 8 8 8 8 6 8 6 6
 6 8 8 8 6 6 6 8 8 6 6 8 8 8 8 8 8 6 8 8 8 6 6 6 8 8 8 6 6 8 6 8 8 6 8 6 8
 6 8 6 8 6 8 8 8 8 8 8 8 8 8 6 8 6 8 8 6 8 6 8 8 8 8 6 8 8 6 8 8 8 8 8 6 8
 6 8 8 8 8 8 8 6 8 8 6 8 6 8 6 8 6 6 8 8 8 8 8 8 8 8 8 8 6 8 8 8 6 6 8 8 8
 8 8 8 8 8 6 8 8 6 8 6 8 8 8 8 8 8 8 6 8 6 8 8 6 8 8 8 8 6 8 8 6 6 6 6 6 6
 6 6 6 8 6 8 8 8 8 6 6 8 6 6 6 6 6 8 6 6 8 8 8 8 8 8 6 8 6 8 8 8 8 8 8 8 8
 8 8 6 8 6 8 8 6 6 8 8 8 6 8 8 8 8 8 8 8 8 8 8 8 6